In [1]:
import os
import clip
import torch
from torchvision.datasets import CIFAR100
from torchvision import datasets, transforms, models

/nethome/bdevnani3/anaconda3/envs/p3/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
# Load the model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load('ViT-B/32', device)

In [4]:
clip.load

<function clip.clip.load(name: str, device: Union[str, torch.device] = 'cpu', jit=True)>

In [21]:
## CIFAR-100 Test

# Download the dataset

cifar100 = CIFAR100(root="/nethome/bdevnani3/raid/data/", download=True, train=False)
text_inputs = torch.cat([clip.tokenize(f"a photo of a {c}") for c in cifar100.classes]).to(device)

top_1_accuracy = 0
top_5_accuracy = 0

per_class_accuracy_top1 = { k:[0,0] for k in range(100)} # correct, total
per_class_accuracy_top5 = { k:[0,0] for k in range(100)} # correct, total

with torch.no_grad():
    text_features = model.encode_text(text_inputs)
    text_features /= text_features.norm(dim=-1, keepdim=True)

for i in range(10000):
    if i%100 ==0:
        print(i)

    # Prepare the inputs
    image, class_id = cifar100[i]
    image_input = preprocess(image).unsqueeze(0).to(device)

    # Calculate features
    with torch.no_grad():
        image_features = model.encode_image(image_input)

    # Pick the top 5 most similar labels for the image
    image_features /= image_features.norm(dim=-1, keepdim=True)
    similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)
    values, indices = similarity[0].topk(5)

    if indices[0] == class_id:
        top_1_accuracy +=1
        per_class_accuracy_top1[class_id][0] +=1

    if class_id in indices:
        top_5_accuracy +=1
        per_class_accuracy_top5[class_id][0] +=1

    per_class_accuracy_top1[class_id][1] +=1
    per_class_accuracy_top5[class_id][1] +=1

Files already downloaded and verified
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900


In [1]:
top_1_accuracy

NameError: name 'top_1_accuracy' is not defined

In [19]:
## Flowers test

flowers = datasets.ImageFolder("/nethome/bdevnani3/raid/data/test")

text_inputs = torch.cat([clip.tokenize(f"a photo of a {c}") for c in flowers.classes]).to(device)

top_1_accuracy = 0
top_5_accuracy = 0

per_class_accuracy_top1 = { k:[0,0] for k in range(102)} # correct, total
per_class_accuracy_top5 = { k:[0,0] for k in range(102)} # correct, total

with torch.no_grad():
    text_features = model.encode_text(text_inputs)
    text_features /= text_features.norm(dim=-1, keepdim=True)

for i in range(len(flowers)):
    if i%100 ==0:
        print(i)

    # Prepare the inputs
    image, class_id = flowers[i]
    image_input = preprocess(image).unsqueeze(0).to(device)

    # Calculate features
    with torch.no_grad():
        image_features = model.encode_image(image_input)

    # Pick the top 5 most similar labels for the image
    image_features /= image_features.norm(dim=-1, keepdim=True)
    similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)
    values, indices = similarity[0].topk(5)

    if indices[0] == class_id:
        top_1_accuracy +=1
        per_class_accuracy_top1[class_id][0] +=1

    if class_id in indices:
        top_5_accuracy +=1
        per_class_accuracy_top5[class_id][0] +=1

    per_class_accuracy_top1[class_id][1] +=1
    per_class_accuracy_top5[class_id][1] +=1

0
100
200
300
400
500
600
700
800


In [ ]:
import torch
model = torch.hub.load('pytorch/vision:v0.9.0', 'resnet50', pretrained=True)

In [26]:
import os
import clip
import torch

import numpy as np
from sklearn.linear_model import LogisticRegression
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR100
from tqdm import tqdm

# Load the model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load('ViT-B/32', device)

# Load the dataset
root = os.path.expanduser("/nethome/bdevnani3/raid/data/")
train = CIFAR100(root, download=True, train=True, transform=preprocess)
test = CIFAR100(root, download=True, train=False, transform=preprocess)


def get_features(dataset):
    all_features = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in tqdm(DataLoader(dataset, batch_size=100)):
            features = model.encode_image(images.to(device))

            all_features.append(features)
            all_labels.append(labels)

    return torch.cat(all_features).cpu().numpy(), torch.cat(all_labels).cpu().numpy()

# Calculate the image features
train_features, train_labels = get_features(train)
test_features, test_labels = get_features(test)

# Perform logistic regression
classifier = LogisticRegression(random_state=0, C=0.316, max_iter=1000, verbose=1)
classifier.fit(train_features, train_labels)

# Evaluate using the logistic regression classifier
predictions = classifier.predict(test_features)
accuracy = np.mean((test_labels == predictions).astype(np.float)) * 100.
print(f"Accuracy = {accuracy:.3f}")

1:44,  2.66s/it]
100%|██████████| 500/500 [22:06<00:00,  2.65s/it]

100%|██████████| 100/100 [04:35<00:00,  2.75s/it]
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
Accuracy = 79.900
/nethome/bdevnani3/anaconda3/envs/p3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  7.3min finished


In [29]:
print(f"Accuracy = {accuracy:.3f}")

Accuracy = 79.900


In [7]:
### Feature Extractor
import torchvision
from torch.utils.data import DataLoader

root = os.path.expanduser("/nethome/bdevnani3/raid/data/")

transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize(
            (0.5071, 0.4865, 0.4409), (0.2673, 0.2564, 0.2762)
        ),
    ]
)

train = CIFAR100(root, download=True, train=True, transform=transform)
train_loader = DataLoader(train, batch_size=100, shuffle=True, num_workers=4)
test = CIFAR100(root, download=True, train=False, transform=transform)
test_loader = DataLoader(test, batch_size=100, shuffle=True, num_workers=4)

model_conv = torchvision.models.resnet50(pretrained=True)

model_conv.eval()

def get_features(data_loader):
    all_features = []
    all_labels = []
    for inps, labels in data_loader:
        with torch.no_grad():
            features = model_conv(inps)

            all_features.append(features)
            all_labels.append(labels)
    return torch.cat(all_features).cpu().numpy(), torch.cat(all_labels).cpu().numpy()

Files already downloaded and verified
Files already downloaded and verified


In [8]:
train_features, train_labels = get_features(train_loader)
test_features, test_labels = get_features(test_loader)

In [10]:
import os
import clip
import torch

import numpy as np
from sklearn.linear_model import LogisticRegression
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR100
from tqdm import tqdm

# Perform logistic regression
classifier = LogisticRegression(random_state=0, C=0.316, max_iter=1000, verbose=1)
classifier.fit(train_features, train_labels)

# Evaluate using the logistic regression classifier
predictions = classifier.predict(test_features)
accuracy = np.mean((test_labels == predictions).astype(np.float)) * 100.
print(f"Accuracy = {accuracy:.3f}")

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
Accuracy = 42.710
/nethome/bdevnani3/anaconda3/envs/p3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 14.3min finished


In [ ]:
print(f"Accuracy = {accuracy:.3f}")

In [2]:
from torchvision.datasets import CIFAR100
cifar100 = CIFAR100(root="/nethome/bdevnani3/raid/data/", download=True, train=False)

Files already downloaded and verified


In [8]:
cifar100.__dict__.keys()

dict_keys(['root', 'transform', 'target_transform', 'transforms', 'train', 'data', 'targets', 'classes', 'class_to_idx'])